In [17]:
import ipywidgets
import IPython.display
import matplotlib.pyplot as plt
import librosa
# homemade stuff
import dataloader
import utilities
import features

PATH = "/home/max/ET-TI/Masterarbeit/mirdata/"  # path to datasets
SAMPLERATE = 22050
data = {} # global dictionary that is needed to access data from callback

def update_dropdown_id_options(*args):
    selected_split = dropdown_split.value
    data["dataset"] = dataloader.MIRDataset("beatles",basepath=PATH, split_nr=selected_split)
    dropdown_id.options = list(data["dataset"].getTrackList().keys())
    dropdown_id.value = list(data["dataset"].getTrackList().keys())[0]

def update_selected_track_id(*args):
    textbox_track_id.value = data["dataset"].getTrackList()[dropdown_id.value]

def on_control_change(change):  
    # change some controls according to updates
    pass
    # plotData(range_slider.value,beat_alignment.value,chroma_type.value)

def loadData(_):
    controls_container.children = [selection]  # hide controls
    audiopath,target = data["dataset"][dropdown_id.value]
    data['target'] = target # ground truth estimation
    data['data_split'] = dropdown_split.value # split for neural networks
    y,sr = librosa.load(audiopath,mono=True,sr=22050)
    data['audiosignal'] = y
    data['timevector'], data['chroma'] = features.crpChroma(y,nCRP=22)
    data['rms'] = features.rms(y)
    # data['downbeats'], data['upbeats'] = features.madmom_beats(audiopath,2)
    data['chroma_complexity'] = [
                    features.sumChromaDifferences(data['chroma']),
                    features.angularDeviation(data['chroma']), # or fifth-width
                    features.flatness(data['chroma']),
                    features.shannonEntropy(data['chroma']),
                    features.negativeSlope(data['chroma']),
                    features.nonSparseness(data['chroma']),
                    features.standardDeviation(data['chroma'])
    ]
    data['interval_categories'] = features.intervalCategories(data['chroma'])

    range_slider.max = int(y.shape[0]/sr)
    controls_container.children = [selection, controls] # show controls 
    
def plotData(_):
    with output:
        IPython.display.clear_output(wait=True)
        fig,ax = plt.subplots(1,2,width_ratios=(10,1))
        utilities.plotComplexityFeatures(ax[0],ax[1],data['timevector'],data['chroma_complexity'])
        ax[0].set_xlim(range_slider.value)
        plt.show()

#Create widgets
output = ipywidgets.Output()
dropdown_dataset = ipywidgets.Dropdown(options=["beatles","RWC"],value = "beatles",description='Dataset:',
                                  layout=ipywidgets.Layout(width='20%'),disabled=False)
dropdown_split = ipywidgets.Dropdown(options=[1, 2, 3, 4, 5, 6, 7],value = 3,description='Split:',
                                  layout=ipywidgets.Layout(width='15%'),disabled=False)
dropdown_id = ipywidgets.Dropdown(description='Track ID:',disabled=False,layout=ipywidgets.Layout(width='20%'))
textbox_track_id = ipywidgets.Text(description='',disabled=True)
button_load = ipywidgets.Button(description='Load Track')
# widgets for selecting data
button_load.on_click(loadData)
selection = ipywidgets.HBox([dropdown_dataset,dropdown_split, dropdown_id,textbox_track_id, button_load])

# widgets for controlling the plot
range_slider = ipywidgets.FloatRangeSlider(value=[0, 20], min=0, max=10, step=1, description='Time Range:', readout_format='.1f')
button_plot = ipywidgets.Button(description='Plot data')
controls = ipywidgets.HBox([range_slider,button_plot])
button_plot.on_click(plotData)
# Initially hide the controls
controls_container = ipywidgets.VBox([selection])

dropdown_split.observe(update_dropdown_id_options, 'value')
dropdown_id.observe(update_selected_track_id, 'value')

range_slider.observe(on_control_change, 'value')

# initialize dropdown menus
update_dropdown_id_options()
update_selected_track_id()
IPython.display.display(controls_container)
IPython.display.display(output)

Output()

In [ ]:

def predictLabels(chroma,key_candidates):

    # up to three chord labels can be estimated
    # calculate angles of triad prototypes for chord estimation
    templates, labels = pitchspace.createChordTemplates()
    dphi_FR = np.zeros((7,))
    dphi_TR = np.zeros((7,))
    # only the template chord of the key c-major is used (pitch_class index=0)
    x_F, x_FR, x_TR, x_DR = pitchspace.transformChroma(templates[0,:,:])
    _, rho_FR, rho_TR, _ = pitchspace.transformChroma(chroma)
    chord_candidates = [] # estimated chord lables

    for time_index in range(chroma.shape[0]):
        temp_chord_candidates =  [None,None,None]
        for i,key in enumerate(key_candidates[time_index]):
            # extract feature in the estimated key for the current timestep
            r_FR = rho_FR[time_index, key]
            r_TR = rho_TR[time_index, key]
            # compute the angle difference for all template chords
            for template in range(7):
                # compute angle difference
                dphi_FR[template] = pitchspace.angle_between(r_FR,x_FR[template,0])
                dphi_TR[template] = pitchspace.angle_between(r_TR,x_TR[template,0])
            # pick the chordprototype with the minimum distance 
            d = np.argmin(dphi_FR+dphi_TR)
            # access the correct label for the current key and chordprototype
            temp_chord_candidates[i] = labels[key][d]
        chord_candidates.append(temp_chord_candidates)

    # Up to 3 candidates are treated separately from now on, we create intervals for them
    est_labels = []  # a nested list holding all estimated chord labels for 3 candidates
    est_intervals = [] # a nested list holding all intervals for the 3 candidates
    t_start = t[0]
    # iterate over all candidates
    for i in range(3):
        candidate_labels = []
        candidate_intervals = []
        # pairwise iteration for all timestep 
        for time,candidates in zip(itertools.pairwise(t),itertools.pairwise(chord_candidates)):
            if(candidates[0][i] != candidates[1][i]): # chord change
                candidate_intervals.append([t_start,time[1]])
                candidate_labels.append(candidates[0][i] )
                t_start = time[1]
            else: # no chord change
                continue
        # append last label
        candidate_intervals.append([t_start,time[1]])
        candidate_labels.append(candidates[1][i])
        # add the result to the list of estimations
        est_intervals.append(np.array(candidate_intervals))
        est_labels.append(candidate_labels)
    return est_intervals, est_labels

Chord estimation with pitch space with a beat segment of the Chromagram

In [ ]:
# split chromagram into beat segments
bar_indices = []
for beat0, beat1 in itertools.pairwise(downbeats):
    if beat0 < start:
        continue
    elif beat1 > stop:
        break
    idx0 = np.where(t >= beat0)[0][0]
    idx1 = np.where(t >= beat1)[0][0]
    bar_indices.append((idx0,idx1))

# select a bar
bar = 0
i_0 = bar_indices[bar][0]
i_1 = bar_indices[bar][1]
chroma_temp = chroma[i_0:i_1]
t_temp = t[i_0:i_1]
rho_F,rho_FR,rho_TR,rho_DR = pitchspace.transformChroma(chroma_temp)

# filter pc energy up to the selected bar!
pc_energy = pitchspace.getPitchClassEnergyProfile(chroma[:i_1],threshold=0.7,angle_weight=0.5)
pc_energy_filtered = pitchspace.filterPitchClassEnergy(pc_energy,alpha=0.96)
keys = np.argsort(pc_energy_filtered[i_0:])
keys = keys[:,-1]  # pick the most likely keys
labels = pitchspace.estimateChordLabels(t_temp, chroma_temp,keys)
est_intervals,est_labels = utilities.createChordIntervals(t_temp,labels)

complexity_features = [
                features.sumChromaDifferences(chroma_temp),
                features.angularDeviation(chroma_temp),
                features.flatness(chroma_temp),
                features.shannonEntropy(chroma_temp),
                features.negativeSlope(chroma_temp),
                features.nonSparseness(chroma_temp),
                features.standardDeviation(chroma_temp)
]
interval_categories = features.intervalCategories(chroma_temp)
# VISUALIZATION 
fig,ax = plt.subplots(4,2,height_ratios=(1,1,7,7),width_ratios=(7,7),figsize=(9,5))
utilities.plotChordAnnotations(ax[0,0],target,(t[i_0],t[i_1]))
ax[0,1].axis("off")
ax[0,1].text(0,0.2,"<- Annotations")
utilities.plotChordAnnotations(ax[1,0],(est_intervals,est_labels),(t[i_0],t[i_1]))
ax[1,1].axis("off")
ax[1,1].text(0,0.2,"<- Predictions")
utilities.plotChromagram(ax[2,0],t[i_0:i_1],chroma[i_0:i_1]);
ax[2,0].set_xlim(t[i_0],t[i_1]);
ax[2,0].set_xlabel("")
ax[2,0].set_xticklabels([])
utilities.plotIntervalCategories(ax[2,1],t_temp,interval_categories)
ax[2,1].set_xlabel("")
ax[2,1].set_xticklabels([])
utilities.plotComplexityFeatures(ax[3,0],ax[3,1],t_temp,complexity_features)
ax[3,0].set_xlabel("Time in s")

current_pitch_class = -1 # initialization
for i in range(t_temp.shape[0]):
    if current_pitch_class != keys[i]:
        current_pitch_class = keys[i]
        fig,ax = plt.subplots(1,4,figsize=(9,9/4))
        pitchspace.plotDiatonicTriads(ax,current_pitch_class)    
    z = rho_F[i] * 1j
    ax[0].plot([z.real], [z.imag], "x",color = utilities.getColor(labels[i]), markersize=5)
    z = rho_FR[i, current_pitch_class] * 1j
    ax[1].plot([z.real], [z.imag], "x",color = utilities.getColor(labels[i]), markersize=5)
    z = rho_TR[i, current_pitch_class] * 1j
    ax[2].plot([z.real], [z.imag], "x",color = utilities.getColor(labels[i]), markersize=5)

    